In [1]:
import spacy as spacy
#check https://towardsdatascience.com/auto-generated-knowledge-graphs-92ca99a81121
from spacy.matcher import Matcher 
from spacy.tokens import Span 
import re
import os.path
my_path = os.path.abspath('')

In [2]:
#read all the text file in the directory
def file_reader(path):
    files = []
    for r,d,f in os.walk(path):
        for file in f:
            if '.txt' in file:
                files.append(os.path.join(r,file))       
    return files

#print elements from list
def print_list(lists):
    for list in lists:
        print(list)

In [3]:
nlp = spacy.load('en_core_web_lg')


files = file_reader(my_path+"/clean_data/item1_business")
files.sort()
print_list(files)




/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/filing_0_section_1.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/filing_10_section_0.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/filing_11_section_0.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/filing_13_section_0.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/filing_14_section_0.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/filing_15_section_0.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/filing_16_section_0.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_bu

In [4]:
text=open(files[6])
print(text.read())
#if \n Blablab \n if the first character is capitalized, add period
#if \n blablab \n if the first character is not capitalized, remove the first \n


ITEM 1. BUSINESS.
GENERAL.
Our vision.
Microsoft is a technology company whose mission is to empower every person and every organization on the planet to achieve more. We strive to create local opportunity, growth, and impact in every country around the world. Our strategy is to build best-in-class platforms and productivity services for an intelligent cloud and an intelligent edge infused with artificial intelligence (“AI”).
The way individuals and organizations use and interact with technology continues to evolve. A person’s experience with technology increasingly spans a multitude of devices and becomes more natural and multi-sensory with voice, ink, and gaze interactions. We believe a new technology paradigm is emerging that manifests itself through an intelligent cloud and an intelligent edge where computing is more distributed, AI drives insights and acts on the user’s behalf, and user experiences span devices with a user’s available data and information. We continue to transfor

In [13]:
text=open(files[6])
doc = nlp(text.read())
#print entities NER
entity_pairs = []
for sent in doc.sents:
    if sent.ents:
        #entity_pairs.append([sent.ents[0],sent.ents[1]])
        print(sent)
        print(sent.ents)
        #print("["+sent.ents[0].text+", "+sent.root.text+", "+sent.ents[1].text+"]" )
        print()
    #if ent.label_=='ORG' or ent.label_=='PRODUCT' or ent.label_=='PERSON':
        #print(ent.text, ent.start_char, ent.end_char, ent.label_)



ITEM 1.
[1]

Microsoft is a technology company whose mission is to empower every person and every organization on the planet to achieve more.
[Microsoft]

Founded in 1975, we operate worldwide in over 190 countries.
[1975, 190]

To achieve our vision, our research and development efforts focus on three interconnected ambitions:.

[three]

Productivity is our first and foremost objective, to enable people to meet and collaborate more easily, and to effectively express ideas in new ways.
[first]

The foundation for these efforts rests on advancing our leading productivity, collaboration, communication, and business process tools including Microsoft Office, Microsoft Dynamics, and LinkedIn.
[Microsoft Office, Microsoft Dynamics, LinkedIn]

With Office 365, we provide familiar industry-leading productivity and business process tools as cloud services, enabling access from anywhere and any device. 
[Office 365]

New scenarios – like those enabled by Microsoft Teams – will redefine how work

[the Surface Studio, December 2016, June 2017]

In July 2015, we announced a plan to restructure our phone business to better focus and align resources.
[July 2015]

In May 2016, we announced plans to further streamline our smartphone hardware business.
[May 2016]

In November 2016, we completed the sale of our feature phone business.

[November 2016]

We released Xbox One and Xbox One S in November 2013 and August 2016, respectively, and announced Xbox One X in June 2017.
[Xbox One, Xbox One S, November 2013, August 2016, Xbox One X, June 2017]

With the launch of the Windows 10 Xbox app in July 2015, and the launch of the Mixer service in May 2017, we continue to open new opportunities for customers to engage both on- and off-console.
[the Windows 10 Xbox, July 2015, May 2017]

Xbox Live enables people to connect and share online gaming experiences and is accessible on Xbox consoles, Windows-enabled devices, and other devices. 
[Xbox Live, Xbox]

Xbox Live is designed to benefit user

In [14]:
#get root dep_ as relations
entities_relation = []
for sent in doc.sents:
    if sent.ents:
        for token in sent:
            if token.pos_=="VERB":
                #entities_relation.append(token.text)
                print(sent)
                print("["+token.text+"]")
                print()

Microsoft is a technology company whose mission is to empower every person and every organization on the planet to achieve more.
[empower]

Microsoft is a technology company whose mission is to empower every person and every organization on the planet to achieve more.
[achieve]

Founded in 1975, we operate worldwide in over 190 countries.
[Founded]

Founded in 1975, we operate worldwide in over 190 countries.
[operate]

To achieve our vision, our research and development efforts focus on three interconnected ambitions:.

[achieve]

To achieve our vision, our research and development efforts focus on three interconnected ambitions:.

[focus]

Productivity is our first and foremost objective, to enable people to meet and collaborate more easily, and to effectively express ideas in new ways.
[enable]

Productivity is our first and foremost objective, to enable people to meet and collaborate more easily, and to effectively express ideas in new ways.
[meet]

Productivity is our first and fo

Skype for Business and Skype also compete with a variety of instant messaging, voice, and video communication providers, ranging from start-ups to established enterprises.
[compete]

Skype for Business and Skype also compete with a variety of instant messaging, voice, and video communication providers, ranging from start-ups to established enterprises.
[ranging]

Skype for Business and Skype also compete with a variety of instant messaging, voice, and video communication providers, ranging from start-ups to established enterprises.
[established]

LinkedIn faces competition from online recruiting companies, talent management companies, and larger companies that are focusing on talent management and human resource services; job boards; traditional recruiting firms; and companies that provide learning and development products and services.
[faces]

LinkedIn faces competition from online recruiting companies, talent management companies, and larger companies that are focusing on talent man

[spanning]

Cloud and Enterprise, focuses on our cloud infrastructure, server, database, CRM, ERP, management and development tools, and other business process applications and services for enterprises.

[focuses]

Devices Group, focuses on our Windows platform, applications, games, store, and devices that power the Windows ecosystem.

[focuses]

Devices Group, focuses on our Windows platform, applications, games, store, and devices that power the Windows ecosystem.

[power]

LinkedIn, focuses on our services that transform the way customers hire, market, sell, and learn.

[focuses]

LinkedIn, focuses on our services that transform the way customers hire, market, sell, and learn.

[transform]

LinkedIn, focuses on our services that transform the way customers hire, market, sell, and learn.

[hire]

LinkedIn, focuses on our services that transform the way customers hire, market, sell, and learn.

[sell]

LinkedIn, focuses on our services that transform the way customers hire, market, se

In [ ]:
for i in range(len(entity_pairs)):
    print("["+entity_pairs[i][0].text+", "+ entities_relation[i]+", "+ entity_pairs[i][1].text+"]")

In [ ]:
#use pos to get relations
#use pos to get non entities objects

pos_chain = "-".join([d.tag_ for d in sent])
    print(sent)
    print(pos_chain)
    if "NNP-VBZ-NNP" in pos_chain: